In [1]:
from pathlib import Path
from IPython.display import HTML, display
import ipywidgets as widgets
import datetime
import pandas as pd
import numpy as np
import re
#print(Path.cwd().parent.parent.parent)
github_repo="https://github.com/taro-ball/thesis-results/"
base_path = Path.cwd().parent.parent.parent / "runs"
#img_path = Path.cwd() / "img.jpg"
# display(Image(filename=img_path))
p = Path(base_path)
#p.as_uri()

In [2]:
tsts = ["k8s_taewa_3lite_","asg_taewa_3lite_", "k8s_apache_3_", "k8s_taewa_3_", "asg_apache_3_", "asg_taewa_3_"]
tsts = tsts + ["asg_taewa_3extra_", "k8s_taewa_3extra_", "asg_raupi_3x_", "k8s_raupi_3_", "k8s_raupi2_3_", "asg_raupi_3","k8s_taewa_6_", "asg_taewa_6_"]
tsts = tsts + ["asg_riwai_3_", "k8s_riwai_3_", "k8s_taewa2_3_", "k8s_apache2_3new", "k8s_apache2_3_", "asg_raupi_3_70_"]
test = widgets.Dropdown(
       options=tsts,
       value='k8s_apache_3_',
       description='Test:')
all = widgets.Checkbox(value=True,
       description='All')
graphs=["QPS_estimatedProcessedBytes.png","QPS_cpuUtilization.png","QPS_groupInServiceCapacity.png"]
date_filter = widgets.Text(
       value='2022.0[3-9]', # use glob pattern http://pymotw.com/2/glob/
       description='Filter', )
box = widgets.HBox([date_filter, all, test ])
display(box)

In [4]:
#print(test._options_values)
tests=[]
if all.value:
    tests=test._options_values;
else:
    tests.append(test.value)

csv_path=p/'o_summary.csv'

# backup old csv in case processing fails
if  csv_path.exists():
    #Path(csv_path).unlink()
    modified=datetime.datetime.fromtimestamp(Path(csv_path).stat().st_mtime).strftime('%m_%d_%Y_%H_%M_%S')
    csv_path.rename(p / ('o_summary.csv.'+modified ))

# loop through tests
for tst in tests:
    print('\n','='*40,f'\nprocessing {tst}\n')
    search=f'*{date_filter.value}*{tst}*'
    nw = datetime.datetime.now()
    count=0
    iList=[]
    #avg_qps_list=[]
    #avg_cpu_list=[]
    #avg_duration_list=[]
    total_err_list=[]

    all_qps_list=[]
    all_cpu_list=[]
    all_duration_list=[]
    all_err_list=[]

    # loop through each test run
    for i in p.glob(search):
        print(i)
        count+=1
        dir = i.name
        platform=dir.split("_")[2]
        if not (i/"csv").exists():
            line = f"<tr><td rowspan='1'><b>No data found, skip: {dir}</b></td></tr>"
            iList.append(line)
            continue

        #================ read test settings from the log file
        log_path = list(i.glob('load*.log'))[0]
        max_pods=0
        with open(log_path, "r") as file:  # the a opens it in append mode
            for l in range(100):
                statement = next(file).strip()
                if m := re.match(".*scaling_minutes=(\d+).*", statement):
                    scaling_minutes=int(m.group(1))
                elif m := re.match(".*warmup_max_threads=(\d+).*", statement):
                    max_threads=int(m.group(1))
                elif m := re.match(".*max_pods=(\d+).*", statement):
                    max_pods=int(m.group(1))
                elif m := re.match(".*_perc=(\d+).*", statement):
                    cpu_target=int(m.group(1))
                elif m := re.match(".*warmup_cycle_sec=(\d+).*", statement):
                    warmup_seconds=int(m.group(1))
                    
        ##================ get json Fortio data ================
        json_paths = (i/"csv").glob('*.json')
        fortio_path=list(json_paths)[0]

        df = pd.read_json(fortio_path, lines=True)
        df['StartTime'] = pd.to_datetime(df['StartTime'])
        df.set_index(['StartTime'],inplace=True)

        # get performance QPS
        perf_df = df[df['Labels'].str.contains('performance-[1-3]', regex=True)]
        #perf_data=perf_df.ActualQPS
        #qps_lst=list(map(int,perf_data))
        qps_lst=list(perf_df.ActualQPS)
        avg_qps=np.mean(qps_lst)
        all_qps_list.extend(qps_lst)
        #avg_qps_list.append(avg_qps)

        # get scaling start times
        scaling_starts_df = df[df['Labels'].str.contains('scaling-.-1$', regex=True)]

        aws_metric0="backendConnectionErrors"
        c_path_err=list((i/"csv").glob('*'+aws_metric0+'.csv'))[0]
        err_df = pd.read_csv(c_path_err, parse_dates=['datetime'], index_col="datetime")

        ## ================ PERFORMANCE CloudWatch data ================
        aws_metric1="cpuUtilization"
        csv_paths = (i/"csv").glob('*'+aws_metric1+'.csv')
        c_path=list(csv_paths)[0]
        mdf = pd.read_csv(c_path, parse_dates=['datetime'], index_col="datetime")
        avg_perfomance_cpu_list=[]
        #print(c_path)
        for index,value in perf_df.iterrows():
            perf_start=index#pd.to_datetime(stime)
            #print(perf_start)
            
            # get 4 max:
            # mmdf=mdf[perf_start:perf_start+ pd.Timedelta(5, "m")]
            # mmmdf=mmdf.nlargest(4, aws_metric1).sort_index()
            # get 3 mid:
            
            mmmdf=mdf[perf_start+ pd.Timedelta(1, "m"):perf_start + pd.Timedelta(4, "m")]
            cpu_list=list(mmmdf[aws_metric1])
            
            average_cpu=np.mean(cpu_list)
            avg_perfomance_cpu_list.append(average_cpu)
            #print(avg_perfomance_cpu_list)
            #print(list(mmmdf.cpuUtilization))
            if value.Labels[-1] == '1':
                first_perf_start=index
                #print(value.Labels)
        all_cpu_list.extend(avg_perfomance_cpu_list)
        avg_cpu=np.mean(avg_perfomance_cpu_list)
        
        #avg_cpu_list.append(avg_cpu)

        # get number of errors from the start of first performance till the end of last scaling
        merr_df=err_df[first_perf_start:first_perf_start + pd.Timedelta(68, "m")] 
        err_list=list(merr_df[aws_metric0])
        total_err_list.append(sum(err_list))
        #print(total_err_list)

        ##================ get json K8s data ================
        #print(i)
        
        max_replicas=99
        k8s_depl_path=i/"k8s-deploy-metrics.json"
        if platform=="k8s":
            k8s_df = pd.read_json(k8s_depl_path, lines=True)
            k8s_df['time'] = pd.to_datetime(k8s_df['time'],format='%d-%m-%Y %H:%M:%S') #26-03-2022 08:44:30
            k8s_df.set_index(['time'],inplace=True)
            k8s_df=k8s_df.tz_localize(tz='UTC') # localize, Pandas hates tz-naive timestamps
            max_replicas=max(k8s_df['replicas'])
        else:
            max_replicas=int(tst.split("_")[2][0])
            max_pods="NA"


        setup=f"cpu={cpu_target} pods={max_pods} treads={max_threads} sc_min={scaling_minutes} warmup={warmup_seconds}"
        ## ================ get json CloudWatch data ================
        aws_metric2="groupInServiceCapacity"
        csv_paths2 = (i/"csv").glob('*'+aws_metric2+'.csv')
        c_path2=list(csv_paths2)[0]
        cwdf2 = pd.read_csv(c_path2, parse_dates=['datetime'], index_col="datetime")

        ## ================ Process scaling data ================
        scaling_durations=[]
        #print(c_path2)
        for index,value in scaling_starts_df.iterrows():
            scale_start=index
            #print(scale_start)
            
            #print(ccwdf2)
            if platform=="k8s":
                kk8s_df=k8s_df[scale_start:scale_start+ pd.Timedelta(scaling_minutes, "m")]
                max_scaled=kk8s_df[kk8s_df['readyReplicas'].ge(max_replicas)]
            elif platform=="asg":
                ccwdf2=cwdf2[scale_start:scale_start+ pd.Timedelta(scaling_minutes, "m")] # only need to look 14 min ahead as it's the length of the scaling run
                max_scaled=ccwdf2[ccwdf2[aws_metric2].ge(max_replicas)]

            # print(ccwdf2.head(6))
            # print(max_scaled.head(2))

            if max_scaled.size > 0:
                cw_reach_max_time=max_scaled.index[0]
                duration = cw_reach_max_time - index
                duration_in_s = duration.total_seconds()
                #minutes = divmod(duration_in_s, 60)[0]
                #print(reach_max_time)
                #s = scaling_starts_df.index.get_loc(cw_reach_max_time, method='nearest')
                #max_minute=scaling_starts_df[reach_max_time]
                
                # print(index)
                # print(cw_reach_max_time)
                scaling_durations.append(duration_in_s)
            else:
                duration_in_s = 0
        avg_duration=np.mean(scaling_durations)
        all_duration_list.extend(scaling_durations)
        
        if np.isnan(avg_duration): avg_duration=0
        #avg_duration_list.append(avg_duration)

        print(f"[{platform} ({max_replicas})] [ CPU:{avg_cpu} | Dur:{avg_duration} | qps:{avg_qps} ]")

        ##================ generate rows ================
        # print(avg_duration_list)
        line = ""
        accent = "<b>" if count==1 else ""
        github_path=f"{github_repo}tree/master/{dir}"
        line = f"""<tr><td rowspan='1'>{count}. test id: <b>{dir}</b> <a href='{github_path}'>[raw data]</a></td><td>run settings: {setup}</td><td></td>
                <td>{accent}Duration s</td><td>{accent}qps</td><td>{accent}CPU %</td><td>{accent}Err</td>
                </tr><tr>"""
        ####====== Images
        for y in graphs:
            #print(i.name)
            img=f'{i.name}/csv/{y}'
            line+=f"<td><img alt='{y}' src='{img}' /></td>"

        ####====== Data

        #data=str(lst)[1:-1].replace(", ", "<br>")
        line+=f"<td>{list(map(round,scaling_durations))}<br><b>{avg_duration:.0f}</b></td>"
        line+=f"<td>{list(map(round,qps_lst))}<br><b>{avg_qps:.0f}</td>"
        line+=f"<td>{[round(x,2) for x in avg_perfomance_cpu_list]}<br><b>{avg_cpu:.2f}</b></td>"
        line+=f"<td><b>{total_err_list[-1]}</b></td>"

        line+="</tr>"
        #line+="</tr><tr><td rowspan='1'>~~~</td></tr>"
        iList.append(line)
    
    # fix durations list
    if not all_duration_list: all_duration_list=[0]

    total=f"""<tr><td>
        <h3>{tst} </h3>samples: <b>{count}</b><br>date generated: {nw}<br></td><td></td><td align="right"><h3>Overall:&nbsp;&nbsp;&nbsp;&nbsp;</h3></td>
        <td>max:<br>{max(all_duration_list):.0f}<br>min:<br>{min(all_duration_list):.0f}<br>mean:<br><b>{np.mean(all_duration_list):.0f}</b><br></td>
        <td>max:<br>{max(all_qps_list):.0f}<br>min:<br>{min(all_qps_list):.0f}<br>mean:<br><b>{np.mean(all_qps_list):.0f}</b><br></td>
        <td>max:<br>{max(all_cpu_list):.2f}<br>min:<br>{min(all_cpu_list):.2f}<br>mean:<br><b>{np.mean(all_cpu_list):.2f}</b><br></td>
        <td>max:<br>{max(total_err_list):.0f}<br>min:<br>{min(total_err_list):.0f}<br>mean:<br><b>{np.mean(total_err_list):.0f}</b><br></td></tr>"""
    iList.append(total)
    iList.insert(0,total)
    imagesList = ''.join(iList)

    ##================ save HTML ================

    html_path=p/f'o_{tst}.html'
    header=f"<!DOCTYPE html><html><head><title>{tst} - Result Explorer</title>"
    header+='<style>table, th, td {border: 1px solid black;font-size: 20px;} body {font: 20px Arial, sans-serif;}</style></head><div style="color:red; font-weight: bold">Please zoom out to see the full table.</div>'


        
    # header+="</tr></table>"
    menuf=p/"o_menu.html"
    # the menu can be manually customized, will be generated if it does not exist
    if not menuf.exists():
    #================ generate menu
        
        menu=''
        def sorter(item):
            """sort by app and nodes"""
            x = ''.join(item.name.split("_")[2:4])
            return x

        menu+="<br><table><tr>"
        files = sorted(p.glob('*.html'), key=sorter)
        for h in files:
            # if h.name.find(f'{tst}')>0:
            #     test_name=f"<b>{h.stem}</b>"
            # else:
            #     test_name=f"{h.stem}"
            test_name=f"{h.stem}"
            if (files.index(h)% 2) == 0:
                menu+=f"\n<td><a href='{h.name}'>{test_name}</a>&nbsp;"
            else:
                menu+=f"<br><a href='{h.name}'>{test_name}</a>&nbsp;</td>"
        menu+="</table>"
        print("saving menu, exists:",menuf.exists() )
        with open(menuf, 'w') as mf:
            mf.write(menu)
    ## external menu:
    mf = open(menuf, "r")
    header+=mf.read()
    mf.close()
    
    header+="<table>"
    footer="</table></html>"

    page= header + imagesList + footer
    a = HTML(page)
    html_src = a.data
    with open(html_path, 'w') as f:
        f.write(html_src)

    csv=""
    csv+=f'{tst},{count},{tst.split("_")[0]},{tst.split("_")[1]},'
    csv+=f'"{list(map(round,all_duration_list))}",{max(all_duration_list):.0f},{np.mean(all_duration_list):.0f},{min(all_duration_list):.0f},'
    csv+=f'"{list(map(round,all_qps_list))}",{max(all_qps_list):.0f},{np.mean(all_qps_list):.0f},{min(all_qps_list):.0f},'
    csv+=f'"{all_cpu_list}",{max(all_cpu_list):.2f},{np.mean(all_cpu_list):.2f},{min(all_cpu_list):.2f},'
    csv+=f'"{total_err_list}",{max(total_err_list):.0f},{np.mean(total_err_list):.0f},{min(total_err_list):.0f},"{setup}"\n'

    with open(csv_path, "a") as csvfile:
        csvfile.write(csv)

    #display(a, metadata=dict(isolated=True))


processing k8s_raupi2_3_

c:\Users\pa\OneDrive\7.UniCode\0thesis-code\runs\2022.05.28_08-55_k8s_raupi2_3_d071
[k8s (3)] [ CPU:66.16019970087005 | Dur:331.966207 | qps:3822.269011980263 ]
